In [36]:
import numpy as np
import scipy.linalg as LA
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import gmres

In [37]:
GE = []
CD = []
TR = []
CG = []
GMRES = []

for n in range(10,16):
    print("="*9+"n={} output".format(n)+"="*9)
    x = np.array([1.0]*n)

    #generate n*n Hilbert matrix
    Hilbert_n = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            Hilbert_n[i,j] = np.reciprocal((i+1)+(j+1)-1+0.0)

    #generate b_n = H_n x
    b_n = Hilbert_n @ x
    
    
    
    ###use gaussian elimination (lu decomposition) to solve the linear system
    p,l,u = LA.lu(Hilbert_n)
    p_inv_b_n = LA.inv(p) @ b_n

    #solve ly = p^{-1} b_n
    y = np.array([0.0]*n)
    y[0] = p_inv_b_n[0]
    for i in range(1,n):
        y[i] = p_inv_b_n[i] - np.dot(l[i],y)

    #solve uw = y
    w = np.array([0.0]*n)
    w[n-1] = y[n-1]/u[n-1,n-1]
    for i in range(1,n):
        w[n-1-i] = (y[n-1-i] - np.dot(u[n-1-i],w))/u[n-1-i,n-1-i]
    print("Gaussian Elimination:{}".format(w))
    
    
    
    ###use cholesky decomposition (LL^T decomposition) to solve the linear system (Hilbert matrix is real-symmetric and positive-definite)
    L = np.zeros((n,n)) + 0.0
    for j in range(n):
        L[j,j] = np.sqrt(Hilbert_n[j,j] - np.sum(L[j][:j]**2))
        for i in range(j+1,n):
            L[i,j] = (Hilbert_n[i,j] - np.dot(L[i],L[j]))/L[j,j]

    #solve Ly = b
    Y = np.array([0.0]*n)
    for i in range(n):
        Y[i] = (b_n[i] - np.dot(L[i],Y))/L[i,i]

    #solve L^T x = y
    W = np.array([0.0]*n)
    for i in range(n):
        W[n-1-i] = (Y[n-1-i] - np.dot(np.transpose(L)[n-1-i],W))/L[n-1-i,n-1-i]
    print("Cholesky Decomposition:{}".format(W))
    
    
    
    ###use Tikhonov regularization to alleviate this problem

    #calculate SVD of Hilbert matrix
    unitary_v,sv,unitary_u_h = LA.svd(Hilbert_n)
    unitary_u = np.transpose(np.conj(unitary_u_h))

    #choose the value of regularization parameter
    alpha = 1e-13

    #use the formular x_\alpha = \sum_{j=1}^r \frac{\mu_j}{\alpha+\mu_j^2}(b,v_j)u_j to solve (\alpha I +A^\ast A) x_\alpha = A^\ast b
    x_alpha = np.array([0.0]*n)
    for j in range(n):
        x_alpha += (sv[j]/(alpha+sv[j]**2)) * np.dot(b_n,unitary_v[:,j])*unitary_u[:,j] 
        #since b_n is real, so we don't need to take conjugate on b_n
    print("Tikhonov Regularization with alpha = {}:{}".format(alpha,x_alpha))    
    
    
    
    ###use conjugate gradient method
    x_cg,exit_code = cg(Hilbert_n,b_n)
    print("Conjugate Gradient:{}".format(x_cg))
    
    
    
    ###use GMRES method
    x_gmres,exit_code = gmres(Hilbert_n,b_n)
    print("GMRES:{}".format(x_gmres))
    
    GE.append(w)
    CD.append(W)
    TR.append(x_alpha)
    CG.append(x_cg)
    GMRES.append(x_gmres)    

=========n=10 output=========
Gaussian Elimination:[1.         0.99999973 1.00000581 0.99994725 1.0002513  0.99930985
 1.00113142 0.99890735 1.0005733  0.99987398]
Cholesky Decomposition:[1.         0.99999979 1.00000435 0.99996074 1.00018631 0.99949006
 1.00083359 0.99919698 1.00042042 0.99990777]
Tikhonov Regularization with alpha = 1e-13:[0.99999895 1.00003359 0.99976634 1.00051595 0.99984201 0.99954939
 0.99990516 1.0003973  1.00042844 0.99956138]
Conjugate Gradient:[0.99905491 1.01004148 0.98356913 0.99197253 1.00497259 1.01246553
 1.01275945 1.00673462 0.99581993 0.98135781]
GMRES:[0.99905712 1.01002942 0.98357323 0.99198118 1.00498007 1.01246976
 1.01275994 1.00673155 0.99581368 0.98134884]
=========n=11 output=========
Gaussian Elimination:[1.00000001 0.9999988  1.00003148 0.99964523 1.00212776 0.99247652
 1.01646108 0.97746392 1.01878886 0.99127853 1.00172782]
Cholesky Decomposition:[1.         1.0000005  0.99998602 1.00016657 0.99895627 1.00382384
 0.99138431 1.01209046 0.989

In [38]:
#print a table of ||x-x_tilde||_2/||x|| for LaTeX
print("\\toprule")
print("~&Gauss消去&Cholesky分解&Tikhonov正则化&共轭梯度法&GMRES法"+"\\\\")
print("\\midrule")
for i in range(6):
    print("$n=${}&{:.4f}&{:.4f}&{:.4f}&{:.4f}&{:.4f}\\\\".format(i+10,(np.sqrt(np.sum((GE[i]-1)**2)))/(np.sqrt(i+10)),\
                                             np.sqrt(np.sum((CD[i]-1)**2))/(np.sqrt(i+10)),\
                                             np.sqrt(np.sum((TR[i]-1)**2))/(np.sqrt(i+10)),\
                                             np.sqrt(np.sum((CG[i]-1)**2))/(np.sqrt(i+10)),\
                                             np.sqrt(np.sum((GMRES[i]-1)**2))/(np.sqrt(i+10))))
print("\\bottomrule")

\toprule
~&Gauss消去&Cholesky分解&Tikhonov正则化&共轭梯度法&GMRES法\\
\midrule
$n=$10&0.0006&0.0004&0.0003&0.0109&0.0109\\
$n=$11&0.0108&0.0058&0.0003&0.0125&0.0125\\
$n=$12&0.2376&0.4648&0.0003&0.0141&0.0141\\
$n=$13&2.5589&0.1976&0.0002&0.0156&0.0156\\
$n=$14&2.1928&25.5365&0.0002&0.0170&0.0170\\
$n=$15&7.5937&139.6530&0.0002&0.0184&0.0184\\
\bottomrule
